In [16]:
import os
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional
from abc import ABC
from datetime import datetime, timezone

# Step 1: Define the BModel as the base class
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 2: Connect to TinyDB and fetch the latest schema
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')

def get_latest_schema():
    """Retrieve the latest schema version from TinyDB."""
    if not schema_table:
        raise ValueError("No schema versions found in TinyDB!")
    
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']


# Step 3: Generate SQLModel class definitions as Python code
def generate_sqlmodel_class_code(table_name: str, columns: list) -> str:
    """Generate Python code for a SQLModel class."""
    class_code = f"from sqlmodel import SQLModel, Field\nfrom typing import Optional\nfrom datetime import datetime, timezone\n\n"
    class_code += f"class {table_name}(BModel):\n"

    for column in columns:
        column_name = column['name']
        column_type = column['type'].upper()
        
        # Map SQLite types to Python/SQLModel types
        if "INT" in column_type:
            field_type = "int"
        elif "TEXT" in column_type:
            field_type = "str"
        elif "REAL" in column_type:
            field_type = "float"
        elif "BLOB" in column_type:
            field_type = "bytes"
        else:
            field_type = "str"  # default to string if unknown type
        
        class_code += f"    {column_name}: Optional[{field_type}] = Field(default=None)\n"

    return class_code


# Step 4: Save the generated classes to Python files
def save_model_files(models: dict, output_dir: str):
    """Save each model as a Python file."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for table_name, model_code in models.items():
        file_path = os.path.join(output_dir, f"{table_name.lower()}.py")
        with open(file_path, "w") as file:
            file.write(model_code)
        print(f"Model file created: {file_path}")


# Step 5: Main workflow
try:
    latest_schema = get_latest_schema()

    # Directory where Python model files will be saved
    output_directory = "models"

    # Generate model code for each table
    models = {}
    for table_name, columns in latest_schema.items():
        models[table_name] = generate_sqlmodel_class_code(table_name, columns)
    
    # Save the models as Python files
    save_model_files(models, output_directory)

except ValueError as e:
    print(e)


Model file created: models/tenant.py
Model file created: models/sqlite_sequence.py
Model file created: models/category.py
Model file created: models/product.py
